In [1]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns

In [3]:
df_transactions_wofirstname = pd.read_csv("C:/Users/felix.oechslein/Desktop/Daten, 13.12.22/Rohdaten/ELPHI_transactions_ohne_vorname.csv")

C:\Users\felix.oechslein\AppData\Local\Temp\ipykernel_12700\2977916244.py:1: DtypeWarning: Columns (13,15,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_transactions_wofirstname = pd.read_csv("C:/Users/felix.oechslein/Desktop/Daten, 13.12.22/Rohdaten/ELPHI_transactions_ohne_vorname.csv")


# Datenbereinigung und Erstellung einer Privatkunden Dataframe

In [25]:
#Transaktions-Daten bereinigen

df_t = df_transactions_wofirstname[["iid", "key", "attrtype", "status", "pricetotal"]]
df_t["pricetotal"] = pd.to_numeric(df_t["pricetotal"], errors = "coerce")
df_t = df_t.query('attrtype == "TI"')
df_t = df_t.query('status == "Buchung" or status == "Order"')
df_t = df_t.query('pricetotal >= 0.01')

# Spalte mit Mean-TIs erstellen
df_t = df_t.groupby(["iid", "key"])["key"].agg("count")
df_t = df_t.to_frame()
df_t = df_t.rename(columns = {"key" : "AnzahlTIsProOrder"})
df_t = df_t.reset_index()

df_t = df_t.groupby("iid")["AnzahlTIsProOrder"].agg("mean")
df_t = df_t.to_frame()
df_t = df_t.rename(columns = {"AnzahlTIsProOrder" : "MeanTIperOR"})
df_t = df_t.reset_index()
df_t.head()

C:\Users\felix.oechslein\AppData\Local\Temp\ipykernel_12700\2983063296.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t["pricetotal"] = pd.to_numeric(df_t["pricetotal"], errors = "coerce")


,iid,MeanTIperOR
0,163094976,2.857143
1,163095007,2.000000
2,163095129,2.000000
3,163095218,2.000000
4,163095249,2.000000


In [26]:
# Computing IQR
Q1 = df_t["MeanTIperOR"].quantile(0.25)
Q3 = df_t["MeanTIperOR"].quantile(0.75)
IQR = Q3 - Q1
IQR_neu = Q3 + 1.5 * IQR
print(IQR_neu)

4.5


In [27]:
#Ausgabe der iids die als Privatkunden klassifiziert werden

df_t = df_t.query("MeanTIperOR <= @IQR_neu")
df_t = df_t["iid"]

In [28]:
#Exportieren der Privatkunden iids

export_csv = df_t.to_csv('privatkunden.iid.ELPHI.csv')